# Parte 1 : Importando Bibliotecas

In [0]:
import csv
import pandas as pd
import numpy as np
import nltk
import re
import collections
import bisect
from nltk.tokenize import RegexpTokenizer
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
from nltk.tokenize import word_tokenize
from nltk import tokenize 
from nltk.stem import RSLPStemmer
from nltk.stem.porter import *
import heapq
import heapq as hp
import time
%matplotlib inline

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


##  Lendo arquivo result.csv

In [0]:
arquivo = pd.read_csv('results.csv')
texto = arquivo['text']

def eliminar_carac(text):
  return re.sub(r'(\W+|\d+|\b\w{0,2}\b)', ' ', text.lower())
texto = texto.apply(eliminar_carac)

texto = texto.apply(tokenize.word_tokenize, language='portuguese')
arquivo.head()

,title,subtitle,author,date,section,text,url
0,“A sociedade foi Rubens Paiva não os facínora...,A decisão da juíza que proíbe as Forças Armada...,F. M.,30/03/2019 00:11:08,Brasil,A juíza federal Ivani Silva da Luz de Brasíli...,https://brasil.elpais.com/brasil/2019/03/26/po...
1,Justiça suspende decisão que proibia Forças Ar...,Liminar havia sido concedida na sexta-feira a ...,Marina Rossi,30/03/2019 16:17:59,Brasil,Menos de 24 horas depois de a juíza federal Iv...,https://brasil.elpais.com/brasil/2019/03/30/po...
2,Governo Bolsonaro prega “negacionismo históric...,Marcos Napolitano professor da USP diz que o...,Regiane Oliveira,04/04/2019 22:37:48,Brasil,Quando determinou que de 31 de março 1964 u...,https://brasil.elpais.com/brasil/2019/04/05/po...
3,Quando os pais de Gabo perceberam que tinham u...,Gustavo Tatis percorre o universo de García Má...,Jesús Ruiz Mantilla,07/03/2019 16:38:56,Cultura,Quando era pequeno Luisa e Gabriel se preo...,https://brasil.elpais.com/brasil/2019/03/06/cu...
4,Rádios canadenses banem músicas de Michael Jac...,Quebec Cogeco Media toma a decisão após queixa...,Jaime Porras Ferreyra,07/03/2019 16:12:37,Cultura,Desde a manhã da última segunda-feira e ...,https://brasil.elpais.com/brasil/2019/03/06/cu...


##Eliminando as stopwords : Usando a biblioteca nltk
Algoritmo Build Index : Usada para uma indexação simples

In [0]:
stopwords = nltk.corpus.stopwords.words('portuguese')

def eliminar_stopwords(text):
  palavras = []
  for palavra in text:
    if(palavra not in stopwords):
      palavras.append(palavra)
  return palavras 

def build_index(arquivo):
  numero_da_lista = 0
  dic_index = {}
  
  for documento in arquivo:
    
    numero_da_lista += 1
    doc = eliminar_stopwords(documento)
    
    for palavra in doc:
      ocorrencia = doc.count(palavra)
      par = (numero_da_lista, ocorrencia)
      if palavra not in dic_index:
        dic_index[palavra] = []
      
      if par not in dic_index[palavra]:
         dic_index[palavra].append(par)
  return dic_index
index = {}              
index = build_index(texto)


# Parte 2 : Usando IDF como complemento 

In [0]:
M = texto.count()
IDF = {}
for pa in index:
  k = len(index[pa])
  idf = round(np.log((M+1)/k),2)
  IDF[pa] = idf

tabela = {'palavra' : list(index.keys()),
       'doc|ocorrencia' : list(index.values()) ,
       'IDF' : list(IDF.values())}
df = pd.DataFrame(tabela)
df.head()


,palavra,doc|ocorrencia,IDF
0,juíza,"[(1, 2), (2, 1)]",4.83
1,federal,"[(1, 2), (2, 2), (3, 1), (6, 1), (7, 3), (15, ...",1.63
2,ivani,"[(1, 1), (2, 1)]",4.83
3,silva,"[(1, 3), (2, 1), (6, 1), (14, 2), (26, 1), (73...",3.04
4,luz,"[(1, 3), (2, 1), (9, 1), (17, 1), (32, 2), (78...",2.53


# Parte 3 :

## 3.1 Representação binária

In [0]:
def binario_r(query, doc):
  contador = 0
  query_palavras = query.split()
  doc_palavras = doc.split()
  
  for palavra in query_palavras:
    contador += (palavra in doc_palavras)
    
  return contador

## 3.2 TF

In [0]:
def tf_fun(query, doc):
  contador = 0
  doc_palavras = doc.split()
  query_palavras = query.split()
  
  for palavra in query_palavras:
    contador += doc_palavras.count(palavra)
  
  return contador

## 3.3 TF-IDF

In [0]:
def tf_idf(query, document):
  score = 0
  doc_tokens = document.split()
  query_tokens = query.split()
  
  for word in query_tokens:
    cwd = doc_tokens.count(word)
    if word in index:
      score += cwd * len(index[word][-1])
  
  return round(score,2)

## 3.4 BM25

In [0]:
def bm25(query, document, k):
  score = 0
  doc_tokens = document.split()
  query_tokens = query.split()
  
  words = [word for word in query_tokens if word in doc_tokens]
    
  for word in words:
    cwd = doc_tokens.count(word)
    dfw = 0
    if word in index:
      dfw = len(index[word][:-1])
    score += (((k+1) * cwd) / (cwd + k)) * np.log10(((M+1) / dfw)) if dfw != 0 else 0
  
  return round(score,2)

# Parte 4 :  3 consultas de ["política", "presidente", "jair bolsonaro"] e seus top-5 documentos mais similares à cada consulta

In [0]:
def top5(query):
  n = 0
  binario = []
  tf = []
  tfidf = []
  b25 = []
  for doc in arquivo.text:
    doc = doc.lower()
    n += 1
    bisect.insort(binario, (binario_r(query, doc), n))
    bisect.insort(tf, (tf_fun(query, doc), n))
    bisect.insort(tfidf, (tf_idf(query, doc), n))
    bisect.insort(b25, (bm25(query, doc,20), n))
 
  binario.reverse()
  tf.reverse()
  tfidf.reverse()
  b25.reverse()
  
  return binario[:5],tf[:5],tfidf[:5],b25[:5]

In [0]:
queries = ["política", "presidente", "jair bolsonaro"]

top5_b = ['','','']
top5_tf = ['','','']
top5_tfidf = ['','','']
top5_bm25 = ['','','']

In [0]:
top5_b[0], top5_tf[0], top5_tfidf[0], top5_bm25[0] = top5(queries[0])
top5_b[1], top5_tf[1], top5_tfidf[1], top5_bm25[1] = top5(queries[1])
top5_b[2], top5_tf[2], top5_tfidf[2], top5_bm25[2] = top5(queries[2])
df = pd.DataFrame()

df['Query'] = queries
df['Binary'] = top5_b
df['TF'] = top5_tf
df['TF IDF'] = top5_tfidf
df['BM25'] = top5_bm25
df

,Query,Binary,TF,TF IDF,BM25
0,política,"[(1, 247), (1, 238), (1, 236), (1, 225), (1, 2...","[(8, 69), (5, 204), (5, 173), (5, 165), (5, 63)]","[(16, 69), (10, 204), (10, 173), (10, 165), (1...","[(2.78, 69), (1.95, 204), (1.95, 173), (1.95, ..."
1,presidente,"[(1, 247), (1, 246), (1, 240), (1, 239), (1, 2...","[(15, 166), (14, 63), (12, 151), (11, 19), (9,...","[(30, 166), (28, 63), (24, 151), (22, 19), (18...","[(3.07, 166), (2.95, 63), (2.69, 151), (2.54, ..."
2,jair bolsonaro,"[(2, 238), (2, 237), (2, 231), (2, 224), (2, 2...","[(41, 207), (37, 151), (31, 166), (19, 19), (1...","[(82, 207), (74, 151), (62, 166), (38, 19), (2...","[(15.8, 207), (12.27, 151), (8.43, 166), (5.92..."


# Parte 5

In [0]:
def pontos_top(top5):
  return [top[0][0] for top in top5]

def documento_top(top5):
  return [top[0][1] for top in top5]

In [0]:
pontos_binario = pontos_top(top5_b)
documentos_binario = documento_top(top5_b)


pontos_tf = pontos_top(top5_tf)
documentos_tf = documento_top(top5_tf)

pontos_tfidf = pontos_top(top5_tfidf)
documento_tfidf = documento_top(top5_tfidf)

pontos_bm25 = pontos_top(top5_bm25)
documento_bm25 = documento_top(top5_bm25)

## Binario

In [0]:
df_binario = pd.DataFrame()
df_binario['Query'] = queries
df_binario['Pontuacao Binaria'] = pontos_binario
df_binario['Documentos'] = documentos_binario

df_binario

,Query,Pontuacao Binaria,Documentos
0,política,1,247
1,presidente,1,247
2,jair bolsonaro,2,238


##TF

In [0]:
df_tf = pd.DataFrame()
df_tf['Query'] = queries
df_tf['Pontuacao TF'] = pontos_tf
df_tf['Documentos'] = documentos_tf

df_tf

,Query,Pontuacao TF,Documentos
0,política,8,69
1,presidente,15,166
2,jair bolsonaro,41,207


##TF-IDF

In [0]:
df_tfidf= pd.DataFrame()
df_tfidf['Query'] = queries
df_tfidf['Pontuacao TF-IDF'] = pontos_tfidf
df_tfidf['Documentos'] = documento_tfidf

df_tfidf

,Query,Pontuacao TF-IDF,Documentos
0,política,16,69
1,presidente,30,166
2,jair bolsonaro,82,207


##BM25

In [0]:
df_bm25 = pd.DataFrame()
df_bm25['Query'] = queries
df_bm25['Pontuacao BM25'] = pontos_bm25
df_bm25['Documentos'] = documento_bm25

df_bm25

,Query,Pontuacao BM25,Documentos
0,política,2.78,69
1,presidente,3.07,166
2,jair bolsonaro,15.80,207


###Conclusão: O modelo  TF-IDF pode ser considerado mais eficiente, pois alcança resultados satisfatório e com uma grande semelhança com  algoritmos com um cálculo mais complexo "BM25".

## 5.2 : índice de Jaccard

In [0]:
def inter_size(a,b):
  count = 0
  for elemento in a:
    if elemento in b:
      count += 1
  return count

def jaccard_index(a,b):
  n_a = len(a)
  n_b = len(b)
  n_ab = inter_size(a,b)
  exp = n_a + n_b - n_ab
  jaccard = n_ab / exp if exp != 0 else 0
  return jaccard

### Todos os documentos

In [0]:
todos_doc = []
todos_doc.append(documentos_binario)
todos_doc.append(documentos_tf)
todos_doc.append(documento_tfidf)
todos_doc.append(documento_bm25)


In [0]:
tabela = [['#', 'Binario', 'TF', 'TF-IDF', 'BM25']]

for i in range(len(todos_doc)):
  linha = []
  for j in range(len(todos_doc)):
    jaccard = jaccard_index(todos_doc[i], todos_doc[j])
    linha.append(round(jaccard,2))
  tabela.append(linha)
  
for i in range(1, len(tabela)):
  tabela[i].insert(0, tabela[0][i])
  
pd.DataFrame(tabela)


,0,1,2,3,4
0,#,Binario,TF,TF-IDF,BM25
1,Binario,1,0,0,0
2,TF,0,1,1,1
3,TF-IDF,0,1,1,1
4,BM25,0,1,1,1


###Percebemos que o modelo binario tem uma diferença enorme entre os outros algoritmos 